In [16]:
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn.functional as F
import numpy as np
import os
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
import random
import numpy as np
# os.environ['CUDA_HOME'] = 'C:\\Users\\heather\\WPy64-31180\\python-3.11.8.amd64\\Lib\\site-packages\\tensorflow\\include\\third_party\\gpus\\cuda'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


path1='C:\\Users\\heather\\Desktop\\Neuromotor\\Data4ML\\tensors.pth'
path='C:\\Users\\heather\\Desktop\\Neuromotor\\Data4ML'
# Load the saved tensors
checkpoint = torch.load(path1)
X_train_tensors = checkpoint['X_train']
X_test_tensors = checkpoint['X_test']
y_train_tensors = checkpoint['y_train']
y_test_tensors = checkpoint['y_test']

In [19]:
# If that doesn't work use numpy files and turn cell into coding
path='C:\\Users\\heather\\Desktop\\Neuromotor\\Data4ML'
X_test = np.load(path+'\\X_train.npy')
X_train = np.load(path+'\\X_test.npy')
y_test = np.load(path+'\\y_train.npy')
y_train = np.load(path+'\\y_test.npy')
X_test2= X_train[:895,:]
y_test2=y_train[:895,:]

X_train_tensors0 = torch.Tensor(X_train)
y_train_tensors = torch.Tensor(y_train)
X_test_tensors0 = torch.Tensor(X_test)
y_test_tensors = torch.Tensor(y_test)


In [21]:
print("Training Shape:", X_train_tensors0.shape, y_train_tensors.shape)
print("Testing Shape:", X_test_tensors0.shape, y_test_tensors.shape) 
print("2nd Testing Shape:", X_test2.shape, y_test2.shape) 

Training Shape: torch.Size([8101, 6, 25, 11]) torch.Size([8101, 6])
Testing Shape: torch.Size([895, 6, 25, 11]) torch.Size([895, 6])
2nd Testing Shape: (895, 6, 25, 11) (895, 6)


In [22]:
X_train_flattened = X_train_tensors0.view(X_train_tensors0.size(0), X_train_tensors0.size(1), -1)
X_test_flattened = X_test_tensors0.view(X_test_tensors0.size(0), X_test_tensors0.size(1), -1)

# Print the shapes to verify
print("Flattened Training Shape:", X_train_flattened.shape)
print("Flattened Testing Shape:", X_test_flattened.shape)

Flattened Training Shape: torch.Size([8101, 6, 275])
Flattened Testing Shape: torch.Size([895, 6, 275])


In [23]:
# I am flattening the features (11 from fft) and channels into 1 dimension
X_train_tensors = torch.reshape(X_train_tensors0, (X_train_tensors0.size(0), X_train_tensors0.size(1), -1))
X_test_tensors = torch.reshape(X_test_tensors0, (X_test_tensors0.size(0), X_test_tensors0.size(1), -1))

In [24]:
X_train_tensors = torch.transpose(X_train_tensors, 1, 2)
X_test_tensors= torch.transpose(X_test_tensors, 1, 2)

print('X_train shape is ', X_train_tensors.shape)

X_train shape is  torch.Size([8101, 275, 6])


In [25]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed = 42
set_seed(seed)

In [27]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("Device name:", torch.cuda.get_device_name(0))
    print("Number of devices:", torch.cuda.device_count())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

PyTorch version: 2.2.0+cpu
CUDA available: False
Device: cpu


In [28]:
def save_checkpoint(model, optimizer, epoch, loss, path):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }
    torch.save(state, path)

def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

In [29]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)  # Add dropout layer
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)  # Apply dropout
        out = self.fc(out[:, -1, :])
        return out

# Define your model
num_classes = y_train_tensors.shape[1]
input_size =  X_train_tensors.shape[2]  # features
hidden_size = 60
num_layers = 2
set_seed(42)

lstm_model = LSTM(num_classes, input_size, hidden_size, num_layers)

In [30]:
def training_loop(n_epochs, lstm, optimizer, criterion, train_loader, val_loader):
    train_losses, val_losses = [], []
    optimizer_history = []

    early_stopping = 10
    best_val_loss = np.inf
    epochs_no_improve = 0
    
    for epoch in range(n_epochs):
        lstm.train()  # Set model to training mode
        train_loss = 0.0
        epoch_counter=0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = lstm(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)  # Accumulate loss
        train_loss /= len(train_loader.dataset)  # Calculate average loss
        train_losses.append(train_loss)
        lstm.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
                for X_val, y_val in val_loader:
                    val_output = lstm(X_val)
                    val_loss += criterion(val_output, y_val).item() * X_val.size(0)
                    val_loss /= len(val_loader.dataset)
                    val_losses.append(val_loss)
        if val_loss:
            if train_loss:
                if epoch_counter==0:
                    print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
                    epoch_counter+=1
        # Save optimizer state dictionary
        optimizer_state_dict = optimizer.state_dict()
        optimizer_history.append(optimizer_state_dict)
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(lstm.state_dict(), 'best_model_lstm.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping:
                print("Early stopping triggered")
                break 
        
        # Save model checkpoint every ten epochs
        if (epoch+1) % 10 == 0:
            save_checkpoint(lstm, optimizer, epoch, val_loss.item(), 'checkpoint.pth')
            np.save('train_losses.npy', np.array(train_losses))
            np.save('val_losses.npy', np.array(val_losses))
            np.save('optimizer_history.npy', np.array(optimizer_history))
    
    return train_losses, val_losses

In [31]:
train_dataset = TensorDataset(X_train_tensors, y_train_tensors)
val_dataset = TensorDataset(X_test_tensors, y_test_tensors)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=6,
    shuffle=False,  # Shuffle for training
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=6,
    shuffle=False,  # Shuffle for training
    num_workers=0,
    pin_memory=torch.cuda.is_available()
)

In [32]:
import warnings
warnings.filterwarnings('ignore')

n_epochs = 40 # 1000 epochs
learning_rate = 0.001


# Define your model
num_classes = y_train_tensors.shape[1]
input_size =  X_train_tensors.shape[2]  # features
hidden_size = 60
num_layers = 6
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001) # optimizer

lstm_model = LSTM(num_classes, input_size, hidden_size, num_layers)

training_loss,val_loss=training_loop(n_epochs=n_epochs,
              lstm=lstm_model,
              optimizer=optimizer,
              criterion=criterion,
              train_loader=train_loader, val_loader=val_loader)


In [58]:
class CNN(nn.Module):
    def __init__(self, num_classes, input_channels=25, time_int=1, features=11):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, input_channels-num_classes, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(input_channels-num_classes, 6, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        
        # Calculate the size of the output from the conv layers to feed into the fully connected layer
        self._to_linear = None
        self.convs(torch.randn(1, input_channels, time_int, features))
        
        self.fc1 = nn.Linear(self._to_linear, num_classes)
        self.fc2 = nn.Linear(num_classes, num_classes)  # Adjusted to match the shape of target tensor

    def convs(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        if self._to_linear is None:
            self._to_linear = x.view(x.size(0), -1).shape[1]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

num_classes = y_train_tensors.shape[1]
input_channels =  X_train_tensors0.shape[2]  

set_seed(42)
time_int =  X_train_tensors0.shape[1] 
features = X_train_tensors0.shape[3] 

cnn_model = CNN(num_classes, input_channels, time_int, features)

RuntimeError: Given input size: (1x1x1). Calculated output size: (1x0x0). Output size is too small

In [59]:
def training_loop_cnn(n_epochs, cnn, optimizer, criterion, train_loader, X_test, y_test):
    train_losses, val_losses = [], []
    early_stopping = 200
    best_val_loss = np.inf
    epochs_no_improve = 0
    
    for epoch in range(n_epochs):
        cnn.train()  # Set model to training mode
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = cnn(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)  # Accumulate loss
        train_loss /= len(train_loader.dataset)  # Calculate average loss
        train_losses.append(train_loss)
        
        # Evaluation on test set (assuming X_test, y_test are tensors)
        cnn.eval()  # Set model to evaluation mode
        with torch.no_grad():
            test_output = cnn(X_test)
            test_loss = criterion(test_output, y_test)
            val_losses.append(test_loss.item())
        
        # Early stopping
        if test_loss < best_val_loss:
            best_val_loss = test_loss
            epochs_no_improve = 0
            torch.save(cnn.state_dict(), 'best_model_cnn.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == early_stopping:
                print("Early stopping triggered")
                break
        
        # Print loss
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {test_loss.item():.4f}') 
        
        # Save model checkpoint
        if (epoch+1) % 10 == 0:
            save_checkpoint(cnn, optimizer, epoch, test_loss.item(), 'checkpoint.pth')
    
    return train_losses, val_losses

In [60]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
train_dataset = TensorDataset(X_train_tensors0, y_train_tensors)

train_losses, val_losses = training_loop_cnn(n_epochs=50, cnn=cnn_model, 
                                        optimizer=optimizer, 
                                        criterion=criterion, 
                                        train_loader=train_dataset, 
                                        X_test=X_test, 
                                        y_test=y_test)

RuntimeError: Given groups=1, weight of size [32, 25, 3, 3], expected input[1, 6, 25, 11] to have 25 channels, but got 6 channels instead

In [ ]:
# After training, load the best checkpoint if needed
best_model.load_state_dict(torch.load(path+'best_model.pth'))

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(2, 1, 1)
plt.plot(val_losses)
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.title('Validation Loss')

plt.subplot(2, 1, 2)
plt.plot(train_losses)
plt.ylabel('Loss')
plt.xlabel('Iteration')
plt.title('Training Loss')

plt.tight_layout()
plt.show()